In [2]:
# import all functions in Utils
# import sys
# sys.path.insert(1, '../')

from Utils import *

# Create folders to save the results
output = 'output'
folder_img = 'IMG'

create_dir(output)
create_dir(folder_img)

n_shots = 8192

The directory output  already exists
The directory IMG  already exists


In [3]:
''' State Preparation'''
#
## ++++++++++++++++++++++++++++++++++ ##
y_class0 = [1, 0]
y_class1 = [0, 1]

x1_train = [3,   -1] ; x1_train = normalize_custom(x1_train) # y2 = '1'
x2_train = [1,    3] ; x2_train = normalize_custom(x2_train) # y1 = '0'
x3_train = [2,   -2] ; x3_train = normalize_custom(x3_train) # y4 = '1'
x4_train = [1.5, 2.5]; x4_train = normalize_custom(x4_train) # y3 = '0'

D = [x1_train, x2_train, x3_train, x4_train]


x_test = [2, 2]; x_test = normalize_custom(x_test)

In [4]:
print(x1_train, x2_train, x3_train, x4_train, x_test)

[(0.9486832980505138+0j), (-0.31622776601683794+0j)] [(0.31622776601683794+0j), (0.9486832980505138+0j)] [(0.7071067811865475+0j), (-0.7071067811865475+0j)] [(0.5144957554275265+0j), (0.8574929257125441+0j)] [(0.7071067811865475+0j), (0.7071067811865475+0j)]


In [5]:
''' Create Circuit '''

c = ClassicalRegister(4)
C1 = QuantumRegister(4, 'c1')
C2 = QuantumRegister(4, 'c2')
C3 = QuantumRegister(4, 'c3')
C4 = QuantumRegister(4, 'c4')

qc = QuantumCircuit(C1, C2, C3, C4, c)
### Initialization ###

x1 = C1[0]; qc.initialize(D[0], [x1]) # '1'
x2 = C2[0]; qc.initialize(D[1], [x2]) # '0'
x3 = C3[0]; qc.initialize(D[2], [x3]) # '1'
x4 = C4[0]; qc.initialize(D[3], [x4]) # '0'

x_test1 = C1[1]; qc.initialize(x_test, [x_test1])
x_test2 = C2[1]; qc.initialize(x_test, [x_test2])
x_test3 = C3[1]; qc.initialize(x_test, [x_test3])
x_test4 = C4[1]; qc.initialize(x_test, [x_test4])

y_test1 = C1[2]
y_test2 = C2[2]
y_test3 = C3[2]
y_test4 = C4[2]

y1 = C1[3]; qc.x( y1 )
y2 = C2[3]
y3 = C3[3]; qc.x( y3 )
y4 = C4[3]

qc.barrier()

'''Computation'''
# C1
qc.h(y_test1)
qc.cswap(y_test1, x_test1, x1)
qc.h(y_test1)
qc.cx(y1, y_test1)

# C2
qc.h(y_test2)
qc.cswap(y_test2, x_test2, x2)
qc.h(y_test2)
qc.cx(y2, y_test2)

# C1
qc.h(y_test3)
qc.cswap(y_test3, x_test3, x3)
qc.h(y_test3)
qc.cx(y3, y_test3)

# C1
qc.h(y_test4)
qc.cswap(y_test4, x_test4, x4)
qc.h(y_test4)
qc.cx(y4, y_test4)
qc.barrier()


# qc.measure(ancilla1, c1)
qc.measure(y_test1, c[0])
qc.measure(y_test2, c[1])
qc.measure(y_test3, c[2])
qc.measure(y_test4, c[3])

In [6]:
r = exec_simulator(qc)
print(r)
save_dict(r, name = output + '/single_cls')

{'1101': 56, '0011': 219, '0001': 2205, '0100': 1370, '1001': 72, '0101': 2124, '0111': 254, '1100': 50, '0110': 152, '1010': 3, '0010': 181, '1000': 52, '1110': 6, '1011': 9, '1111': 12, '0000': 1427}


In [7]:
### Connection
print(qc)

         ┌──────────────────────────────┐ ░                    ░             
c1_0: |0>┤ Initialize(0.94868,-0.31623) ├─░───────X────────────░─────────────
         ├─────────────────────────────┬┘ ░       │            ░             
c1_1: |0>┤ Initialize(0.70711,0.70711) ├──░───────X────────────░─────────────
         └─────────────────────────────┘  ░ ┌───┐ │ ┌───┐┌───┐ ░ ┌─┐         
c1_2: |0>─────────────────────────────────░─┤ H ├─■─┤ H ├┤ X ├─░─┤M├─────────
                      ┌───┐               ░ └───┘   └───┘└─┬─┘ ░ └╥┘         
c1_3: |0>─────────────┤ X ├───────────────░────────────────■───░──╫──────────
         ┌────────────┴───┴────────────┐  ░                    ░  ║          
c2_0: |0>┤ Initialize(0.31623,0.94868) ├──░───────X────────────░──╫──────────
         ├─────────────────────────────┤  ░       │            ░  ║          
c2_1: |0>┤ Initialize(0.70711,0.70711) ├──░───────X────────────░──╫──────────
         └─────────────────────────────┘  ░ ┌───┐ │ ┌───┐┌───┐ ░

In [ ]:
qc_t1 = transpile([qc1]*1000, backend_b,optimization_level=1)
qc_t2 = transpile([qc]*1000, backend_b,optimization_level=2)
qc_t3 = transpile([qc]*1000, backend_b,optimization_level=3)

In [ ]:
qf_ncx1 = [c.count_ops()['cx'] for c in qc_t1]
qf_ncx2 = [c.count_ops()['cx'] for c in qc_t2]
qf_ncx3 = [c.count_ops()['cx'] for c in qc_t3]

In [ ]:
# check the number of cx gates
print('Opt of lv 1: Minimum # of cx gates = %s' % min(qf_ncx1))
print('Opt of lv 1: The best circuit is the circut %s' % qf_ncx1.index(min(qf_ncx1)))

# check the number of cx gates
print('Opt of lv 2: Minimum # of cx gates = %s' % min(qf_ncx2))
print('Opt of lv 2: The best circuit is the circut %s' % qf_ncx2.index(min(qf_ncx2)))

# check the number of cx gates
print('Opt of lv 3: Minimum # of cx gates = %s' % min(qf_ncx3))
print('Opt of lv 3: The best circuit is the circut %s' % qf_ncx3.index(min(qf_ncx3)))

In [ ]:
opt_qc = qc_t3[qf_ncx3.index(min(qf_ncx3))]

In [ ]:
opt_qc

In [ ]:
qc_DIR

In [ ]:
qc_OPT

In [ ]:
job = execute(opt_qc, backend_b, shots=8192)

In [ ]:
job.status()

In [ ]:
job.job_id()